In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data Load, preprocessing

In [7]:
# 데이터 불러오기
train = pd.read_csv('/Users/yukjunghun/Desktop/coding/git_repository/MachineLearning/10. Machine Learning Competitions/따릉이/train.csv')# 모델 학습 파일
test = pd.read_csv('/Users/yukjunghun/Desktop/coding/git_repository/MachineLearning/10. Machine Learning Competitions/따릉이/test.csv')# 모델 시험지 파일
# submission = pd.read_csv('따릉이/submission.csv')#답안지 파일 

In [8]:
train['hour_bef_temperature'] = train['hour_bef_temperature'].fillna(value = train['hour_bef_temperature'].mean())
train_isna_sum = train.isna().sum()
train_isna_sum[train_isna_sum != 0].index
na_columns = train_isna_sum[train_isna_sum != 0].index
def fill_bicycle_na(df, column) :
    df[column] = df[column].fillna(value = df[column].mean())
fill_bicycle_na(train, 'hour_bef_precipitation')


In [9]:
for col in na_columns:
    fill_bicycle_na(train, col)
    print(col, '결측값 대체 완료')

hour_bef_precipitation 결측값 대체 완료
hour_bef_windspeed 결측값 대체 완료
hour_bef_humidity 결측값 대체 완료
hour_bef_visibility 결측값 대체 완료
hour_bef_ozone 결측값 대체 완료
hour_bef_pm10 결측값 대체 완료
hour_bef_pm2.5 결측값 대체 완료


In [10]:
test_isna_sum = test.isna().sum()
test_na_columns = test_isna_sum[test_isna_sum != 0].index
for col in test_na_columns:
    print(col)
for col in test_na_columns:
    fill_bicycle_na(test, col)
    print(col, '대체 완료')
test.isna().sum()

hour_bef_temperature
hour_bef_precipitation
hour_bef_windspeed
hour_bef_humidity
hour_bef_visibility
hour_bef_ozone
hour_bef_pm10
hour_bef_pm2.5
hour_bef_temperature 대체 완료
hour_bef_precipitation 대체 완료
hour_bef_windspeed 대체 완료
hour_bef_humidity 대체 완료
hour_bef_visibility 대체 완료
hour_bef_ozone 대체 완료
hour_bef_pm10 대체 완료
hour_bef_pm2.5 대체 완료


id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
dtype: int64

In [13]:
train_data = train.drop(columns=['count'], axis=1)
train_target = train['count']
test_data = test

In [89]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_validate

rdforest = RandomForestRegressor(oob_score=True, random_state=42, n_estimators = 500)

In [90]:
rdforest.fit(train_data, train_target)

RandomForestRegressor(n_estimators=500, oob_score=True, random_state=42)

In [91]:
rdforest.oob_score_

0.7928946390498043

In [92]:
np.column_stack((rdforest.feature_names_in_, rdforest.feature_importances_))

array([['id', 0.0258589714624019],
       ['hour', 0.589597022699878],
       ['hour_bef_temperature', 0.18040141112204436],
       ['hour_bef_precipitation', 0.02090437392456853],
       ['hour_bef_windspeed', 0.026522461726435877],
       ['hour_bef_humidity', 0.03389783766499509],
       ['hour_bef_visibility', 0.03316870859621336],
       ['hour_bef_ozone', 0.03733900755389364],
       ['hour_bef_pm10', 0.030667903572284935],
       ['hour_bef_pm2.5', 0.021642301677284394]], dtype=object)

In [93]:
scores = cross_validate(rdforest, train_data, train_target, return_train_score=True)
scores['test_score'].mean()

0.778839097738456

In [94]:
extree = ExtraTreesRegressor(n_estimators=5000, random_state=42)
extree.fit(train_data, train_target)

ExtraTreesRegressor(n_estimators=5000, random_state=42)

In [95]:
scores = cross_validate(extree, train_data, train_target, return_train_score=True)

In [96]:
scores

{'fit_time': array([8.31326199, 8.18947506, 8.28203988, 8.33538222, 8.31877375]),
 'score_time': array([0.48474383, 0.48504496, 0.30299497, 0.41413283, 0.38107514]),
 'test_score': array([0.78710974, 0.78980197, 0.82692799, 0.76547543, 0.81024182]),
 'train_score': array([1., 1., 1., 1., 1.])}

In [97]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(random_state=42, n_estimators=5000, learning_rate=0.05)
gb.fit(train_data, train_target)

GradientBoostingRegressor(learning_rate=0.05, n_estimators=5000,
                          random_state=42)

In [98]:
scores = cross_validate(gb, train_data, train_target, return_train_score=True)

In [99]:
scores

{'fit_time': array([6.05412364, 6.06286597, 6.07003903, 6.05621004, 6.07351518]),
 'score_time': array([0.01480126, 0.01321483, 0.01326203, 0.01311707, 0.013515  ]),
 'test_score': array([0.66817995, 0.75395482, 0.80398449, 0.76126184, 0.72661248]),
 'train_score': array([0.99932832, 0.99946216, 0.99944149, 0.99953962, 0.99937739])}

In [100]:
scores['test_score'].mean()

0.7427987164544152

In [101]:
from sklearn.ensemble import StackingRegressor
# rdforest = RandomForestRegressor(n_estimators=300, random_state=42)
# extree = ExtraTreesRegressor(n_estimators=1000, random_state=42)
sr = StackingRegressor(estimators=(rdforest, extree))

In [104]:
sr.score(train_data, test_data)

AttributeError: This 'StackingRegressor' has no attribute 'predict'